In [12]:
import requests
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import *
from bs4 import BeautifulSoup as bs
from selenium import webdriver 

In [13]:
driver = webdriver.Firefox(executable_path= "geckodriver.exe")
driver.maximize_window()
driver.get('https://www.marketwatch.com/investing/stock/TSLA/financials/cash-flow')
table = driver.find_element_by_id("maincontent").get_attribute('innerHTML')
cashflow  = pd.read_html(table)
cashflow

[   Fiscal year is January-December. All values USD millions.       2015  \
 0                   Net Income before Extraordinaries         (888.66M)   
 1                                   Net Income Growth                 -   
 2              Depreciation, Depletion & Amortization           422.59M   
 3                          Depreciation and Depletion                 -   
 4                   Amortization of Intangible Assets                 -   
 5              Deferred Taxes & Investment Tax Credit                 -   
 6                                      Deferred Taxes                 -   
 7                               Investment Tax Credit                 -   
 8                                         Other Funds           434.86M   
 9                               Funds from Operations          (31.21M)   
 10                                    Extraordinaries                 -   
 11                         Changes in Working Capital         (493.29M)   
 12         

In [14]:
def get_financial_report(ticker):
#     driver = webdriver.Firefox(executable_path= "geckodriver.exe")
#     driver.maximize_window()
#     driver.get('https://www.marketwatch.com/investing/stock/'+ticker+'/financials/cash-flow')
#     table = driver.find_element_by_id("mw-content-text").get_attribute('innerHTML')
#     cashflow  = pd.read_html(table)
#build URLs
    urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/cash-flow'
    urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'
#request the data using beautiful soup
    text_soup_financials = bs(requests.get(urlfinancials).text,"html") 
    text_soup_balancesheet = bs(requests.get(urlbalancesheet).text,"html")
    # build lists for Income statement
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
    epslist=[]
    netincomelist = []
    longtermdebtlist = [] 
    interestexpenselist = []
    ebitdalist= []
    #load data into lists if the row title is found
    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in  title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    # find the table headers for the Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
    equitylist=[]
    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    #get the data from the income statement lists 
    #use helper function get_element
    eps = get_element(epslist,0)
    epsGrowth = get_element(epslist,1)
    netIncome = get_element(netincomelist,0)
    shareholderEquity = get_element(equitylist,0)
    roa = get_element(equitylist,1)
    longtermDebt = get_element(longtermdebtlist,0)
    interestExpense =  get_element(interestexpenselist,0)
    ebitda = get_element(ebitdalist,0)

    # load all the data into dataframe 
    fin_df= pd.DataFrame({'eps': eps,'eps Growth': epsGrowth,'net Income': netIncome,'shareholder Equity': shareholderEquity,'roa': roa,'longterm Debt': longtermDebt,'interest Expense': interestExpense,'ebitda': ebitda}, index=range(date.today().year-5,date.today().year))

    fin_df.reset_index(inplace=True)

    return fin_df

In [15]:
                         
#helper function
def get_element(list,element):
    try:
        return list[element]
    except:
        return '-'
            

In [16]:
get_financial_report('MSFT')

,index,eps,eps Growth,net Income,shareholder Equity,roa,longterm Debt,interest Expense,ebitda
0,2015,-,-,-,72B,37.21%,41.32B,-,-
1,2016,-,-,-,87.71B,35.04%,83.87B,-,-
2,2017,-,-,-,82.72B,31.96%,81.94B,-,-
3,2018,-,-,-,102.33B,35.71%,79.11B,-,-
4,2019,-,-,-,118.3B,39.26%,76.21B,-,-
